In [19]:
import os
import sys
from annoy import AnnoyIndex
import pandas as pd
import numpy as np

In [20]:
debug=False
doTest=False
nrEventToDo=1
nrBucketToDo=2
bucketSize=20

In [21]:
#inputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/TrackML/input/ttbar_mu200-generic"
inputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/data/TrackML/ttbar_mu200-generic"
outputFolderName="/Users/luizaadelinaciucu/Work/ATLAS/TrackML/output2"

In [22]:
def get_eventNumber_from_fileName(fileName):
    eventNumber=fileName.replace("event","").replace("-hits.csv","")
    return eventNumber

In [23]:
list_eventNumber=[]
for fileName in sorted(os.listdir(inputFolderName)):
    if fileName.endswith("hits.csv"):
        print(fileName)
        eventNumber=get_eventNumber_from_fileName(fileName)
        print(eventNumber)
        list_eventNumber.append(eventNumber)
print("list_eventNumber",list_eventNumber)
        


event000000000-hits.csv
000000000
event000000001-hits.csv
000000001
event000000002-hits.csv
000000002
event000000003-hits.csv
000000003
event000000004-hits.csv
000000004
event000000005-hits.csv
000000005
event000000006-hits.csv
000000006
event000000007-hits.csv
000000007
event000000008-hits.csv
000000008
event000000009-hits.csv
000000009
event000000010-hits.csv
000000010
event000000011-hits.csv
000000011
event000000012-hits.csv
000000012
event000000013-hits.csv
000000013
event000000014-hits.csv
000000014
event000000015-hits.csv
000000015
event000000016-hits.csv
000000016
event000000017-hits.csv
000000017
event000000018-hits.csv
000000018
event000000019-hits.csv
000000019
event000000020-hits.csv
000000020
event000000021-hits.csv
000000021
event000000022-hits.csv
000000022
event000000023-hits.csv
000000023
event000000024-hits.csv
000000024
event000000025-hits.csv
000000025
event000000026-hits.csv
000000026
event000000027-hits.csv
000000027
event000000028-hits.csv
000000028
event000000029

In [6]:
def buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False): 
    numberDimension=nparray_position.shape[1] # 3 (x,y,z)
    if debug:
        print("numberDimension",numberDimension)
    index=AnnoyIndex(numberDimension,metric)
    if debug:
        print("type(index)",type(index))
        print("enumerate data")
    # add each hit to the index
    for i,position in enumerate(nparray_position):
        if debug:
            print("i",i,"position",position)
        index.add_item(i,position)
    # done for loop over hits
    # build the index with 10 trees
    index.build(ntrees) # 10 trees
    return index
# done function

In [7]:
def get_input_output_from_df_hits(df_hits,index):
    list_nparray_input=[]
    list_nparray_output=[]
    
    nparray_volume_id=df_hits["volume_id"].values
    nparray_layer_id=df_hits["layer_id"].values
    counter=0
    
    for i in range(len(df_hits)):
        if doTest==True and counter>(nrBucketToDo-1):
            continue
        if (nparray_volume_id[i]==8 and nparray_layer_id[i]==2)==False:
            continue
        counter+=1
        if debug==True:
            print(i)
            
        # using annoy to find the 20 nearest neighboring hits by angle to this hit
        list_index=index.get_nns_by_item(i,bucketSize)
        if debug:
            print("list_index",list_index)
            
        # create bucket
        df_bucket=df_hits.iloc[list_index]
        
        # create NN input for this one bucket
        nparray_input=df_bucket[["x","y","z"]].values.flatten()
        if debug:
            print("nparray_input",nparray_input,"shape",nparray_input.shape,"type",type(nparray_input))
            
        # create NN output for this one bucket
        # identify particle with the largest number of hits in this bucket
        nparray_particle_id=df_bucket["particle_id"].values
        if debug:
            print("nparray_particle_id",nparray_particle_id,"shape",nparray_particle_id.shape,"type",type(nparray_particle_id))
        dict_particle_id_counter={}
        for particle_id in nparray_particle_id:
            if particle_id not in dict_particle_id_counter:
                dict_particle_id_counter[particle_id]=1
            else:
                dict_particle_id_counter[particle_id]+=1
        if debug:
            print("dict_particle_id_counter",dict_particle_id_counter)
            
        # find the maximum value of the counters
        particle_id_with_max_hits=0
        counter_max=0
        for particle_id in dict_particle_id_counter:
            counter=dict_particle_id_counter[particle_id]
            if counter>counter_max:
                counter_max=counter
                particle_id_with_max_hits=particle_id
        if debug:
            print("particle_id_with_max_hits",particle_id_with_max_hits)
            
        # create nparray_output
        list_output=[]
        for particle_id in nparray_particle_id:
            if particle_id==particle_id_with_max_hits:
                list_output.append(1)
            else:
                list_output.append(-1)
        if debug:        
            print("list_output",list_output)
        nparray_output=np.array(list_output)
        if debug:
            print("nparray_output",nparray_output)
            
        if debug and doTest:
            plt.plot(df_bucket.x,df_bucket.y,"o",color="red")
            plt.plot(0,0,"r+")
            plt.title(str(i))
            plt.show()
            # plt.savefig(outputFolderName+"/bucket_"+str(i)+"_"+eventNumber+".pdf")
            
        list_nparray_input.append(nparray_input)
        list_nparray_output.append(nparray_output)
    # done for loop
    
    nparray_input_all=np.array(list_nparray_input)
    nparray_output_all=np.array(list_nparray_output)
    
    return nparray_input_all, nparray_output_all
# done function      

In [8]:
for i,eventNumber in enumerate(list_eventNumber):
    
    if doTest and i>=nrEventToDo:
        continue
        
    print(" ")
    print("**** start i",i,"eventNumber",eventNumber,"*****")
    
    inputFileName_hits_recon=inputFolderName+"/event"+eventNumber+"-hits.csv"
    inputFileName_hits_truth=inputFolderName+"/event"+eventNumber+"-truth.csv"

    fileNameNNInput=outputFolderName+"/NN_1_data_Input_"+eventNumber+".npy"
    fileNameNNOutput=outputFolderName+"/NN_1_data_Output_"+eventNumber+".npy"

    fileNameNNInputTrain=outputFolderName+"/NN_2_data_Input_Train_"+eventNumber+".npy"
    fileNameNNOutputTrain=outputFolderName+"/NN_2_data_Output_Train_"+eventNumber+".npy"

    fileNameNNInputTest=outputFolderName+"/NN_2_data_Input_Test_"+eventNumber+".npy"
    fileNameNNOutputTest=outputFolderName+"/NN_2_data_Output_Test_"+eventNumber+".npy"
    
    df_hits_recon=pd.read_csv(inputFileName_hits_recon)
    df_hits_truth=pd.read_csv(inputFileName_hits_truth)
    df_hits=pd.concat([df_hits_recon,df_hits_truth],axis=1,sort=False)
    # print(df_hits.head())
    
    nparray_position=df_hits[["x","y","z"]].values
    print("nparray_position",nparray_position)
    index=buildAnnoyIndex(nparray_position,metric="angular",ntrees=10,debug=False)
    
    nparray_input_all,nparray_output_all=get_input_output_from_df_hits(df_hits,index)
    print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))
    print("nparray_output_all",nparray_output_all,"shape",nparray_output_all.shape,"type",type(nparray_output_all))
    
    # we will want later to split in half for train and test
    # so keep only an even number
    # if an odd number of buckets, remove the last one
    nrBuckets=nparray_input_all.shape[0]
    if nrBuckets%2==1:
        # odd number of buckets
        # rewrite the input as to remove the last element
        nparray_input_all=nparray_input_all[0:-1]
        # rewrite the output as to remove the last element
        nparray_output_all=nparray_output_all[0:-1]
    else:
        # if even, do nothing
        pass
    # done if
    print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))
    print("nparray_output_all",nparray_output_all,"shape",nparray_output_all.shape,"type",type(nparray_output_all))
    
    # reshape for the input to have an extra dimension of size one, needed by Keras
    nparray_input_all=nparray_input_all.reshape(nparray_input_all.shape[0],nparray_input_all.shape[1],1)
    print("nparray_input_all",nparray_input_all,"shape",nparray_input_all.shape,"type",type(nparray_input_all))
    
    # save them to files
    np.save(fileNameNNInput,nparray_input_all)
    np.save(fileNameNNOutput,nparray_output_all)
    
    # split each of input and output by half in train and test
    nrBucket=nparray_input_all.shape[0] # is now an even number
    # create list of even indices to be used for train
    list_index_Train=[i for i in range(nrBucket) if i%2==0]
    # create list of odd indices to be used for train
    list_index_Test=[i for i in range(nrBucket) if i%2==1]

    # split input in Train and Test by giving as index a list of indices
    nparray_Input_Train=nparray_input_all[list_index_Train]
    nparray_Input_Test=nparray_input_all[list_index_Test]
    nparray_Output_Train=nparray_output_all[list_index_Train]
    nparray_Output_Test=nparray_output_all[list_index_Test]

    # save them to files
    np.save(fileNameNNInputTrain,nparray_Input_Train)
    np.save(fileNameNNInputTest,nparray_Input_Test)
    np.save(fileNameNNOutputTrain,nparray_Output_Train)
    np.save(fileNameNNOutputTest,nparray_Output_Test)
# done for loop 

 
**** start i 0 eventNumber 000000000 *****
nparray_position [[ 4.26321e+01  1.62488e+00 -1.50250e+03]
 [ 4.72534e+01  3.84431e+00 -1.50250e+03]
 [ 8.20138e+01  9.53913e+00 -1.50250e+03]
 ...
 [-8.47474e+02  1.96053e+02  2.95550e+03]
 [-8.99313e+02  4.55782e+01  2.95250e+03]
 [-9.69219e+02  9.38630e+01  2.95250e+03]]
nparray_input_all [[  -32.3306     -4.28872  -462.092   ...   -89.3       -15.1782
  -1302.5    ]
 [  -30.0484    -10.8162   -427.981   ...   -47.7968    -14.5214
   -702.     ]
 [  -30.4775     -9.58902  -421.256   ...  -112.212     -40.1146
  -1498.     ]
 ...
 [  -31.3479      2.50689   469.019   ...   -52.6152      1.65826
    817.5    ]
 [  -29.3948     12.5632    406.38    ...   -90.373      34.3218
   1298.     ]
 [  -30.8299     11.6898    435.472   ...   -74.9913     32.0636
   1098.     ]] shape (11993, 60) type <class 'numpy.ndarray'>
nparray_output_all [[-1  1 -1 ... -1 -1 -1]
 [ 1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 ...
 [-1 -1 -1 ...  1  1  1]
 [ 1

nparray_position [[ 1.02126e+02  2.04404e+00 -1.50250e+03]
 [ 3.41398e+01  5.55290e-01 -1.50250e+03]
 [ 4.44923e+01  5.23195e+00 -1.50250e+03]
 ...
 [-1.00504e+03  2.13293e+01  2.95250e+03]
 [-9.26789e+02  1.05742e+02  2.95250e+03]
 [-8.70917e+02  2.53786e-02  2.95250e+03]]
nparray_input_all [[-3.34237e+01 -1.16228e+00 -4.85865e+02 ... -4.73806e+01 -7.94887e-01
  -7.02500e+02]
 [-3.17646e+01 -5.90755e+00 -4.87369e+02 ... -5.95524e+01 -1.10314e+01
  -9.62500e+02]
 [-3.35555e+01 -7.85547e-01 -4.77808e+02 ... -4.06138e+01 -6.90610e-01
  -5.97500e+02]
 ...
 [-3.15819e+01 -1.64052e+00  4.85879e+02 ... -3.83472e+01 -5.68068e-01
   5.97500e+02]
 [-3.07577e+01  1.29706e+01  4.69442e+02 ... -9.83814e+01  3.84932e+01
   1.49800e+03]
 [-3.14058e+01  1.48017e+00  4.31334e+02 ... -6.02551e+01 -4.75853e-02
   8.17500e+02]] shape (12928, 60) type <class 'numpy.ndarray'>
nparray_output_all [[ 1  1  1 ... -1  1 -1]
 [ 1  1  1 ... -1  1  1]
 [ 1  1 -1 ... -1 -1 -1]
 ...
 [-1  1  1 ...  1  1 -1]
 [-1 -1 

nparray_input_all [[ -32.7351    -3.13191 -442.867   ...  -44.4534    -1.76822 -602.5    ]
 [ -37.2866   -12.1141  -598.      ...  -44.0492   -13.5834  -698.     ]
 [ -30.5373    -9.41792 -480.159   ...  -42.913    -13.9791  -698.     ]
 ...
 [ -28.7558    12.7489   453.941   ...  -36.3047    14.632    598.     ]
 [ -30.7156    13.7165   473.768   ... -103.475     43.5296  1498.     ]
 [ -31.1939     5.23709  439.216   ...  -63.6257     8.74397  957.5    ]] shape (10917, 60) type <class 'numpy.ndarray'>
nparray_output_all [[-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ...  1  1  1]
 [ 1  1  1 ... -1 -1 -1]
 ...
 [ 1  1  1 ... -1 -1 -1]
 [-1  1  1 ... -1 -1 -1]
 [-1  1  1 ... -1 -1 -1]] shape (10917, 20) type <class 'numpy.ndarray'>
nparray_input_all [[ -32.7351    -3.13191 -442.867   ...  -44.4534    -1.76822 -602.5    ]
 [ -37.2866   -12.1141  -598.      ...  -44.0492   -13.5834  -698.     ]
 [ -30.5373    -9.41792 -480.159   ...  -42.913    -13.9791  -698.     ]
 ...
 [ -29.7822    10.6189   45

nparray_position [[   34.0879      5.51651 -1502.5    ]
 [   30.4625      4.98041 -1502.5    ]
 [   42.4802      4.27113 -1502.5    ]
 ...
 [ -952.578     200.1      2955.5    ]
 [ -871.166      -3.92681  2952.5    ]
 [ -922.792       3.64978  2952.5    ]]
nparray_input_all [[-3.34243e+01 -1.16080e+00 -4.22990e+02 ... -1.13684e+02  5.25224e-01
  -1.50200e+03]
 [-1.09252e+02 -2.06670e+01 -1.49800e+03 ... -4.48832e+01 -6.71720e+00
  -6.02500e+02]
 [-1.13401e+02 -1.03733e+01 -1.50200e+03 ... -4.48832e+01 -6.71720e+00
  -6.02500e+02]
 ...
 [-3.14803e+01  1.60623e-01  4.74692e+02 ... -8.57190e+01 -2.60847e+00
   1.30250e+03]
 [-3.06876e+01  1.42129e+01  4.44426e+02 ... -5.06234e+01  2.07795e+01
   6.98000e+02]
 [-3.15422e+01 -9.37631e-01  4.20860e+02 ... -5.82680e+01 -4.41025e+00
   8.22500e+02]] shape (9182, 60) type <class 'numpy.ndarray'>
nparray_output_all [[-1 -1 -1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 ...
 [-1  1  1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [

nparray_position [[   56.2377     10.8709  -1502.5    ]
 [   87.8783     10.3016  -1502.5    ]
 [   75.2433     13.119   -1502.5    ]
 ...
 [ -936.239     174.749    2955.5    ]
 [ -902.414      -3.71995  2952.5    ]
 [ -930.368      48.8547   2952.5    ]]
nparray_input_all [[  -30.1474    -10.533    -429.498   ...   -53.1628    -14.7958
   -702.     ]
 [  -31.8136     -5.76748  -466.794   ...   -68.8521    -11.1615
   -962.5    ]
 [  -31.6424     -6.25717  -458.624   ...   -78.6162    -12.7293
  -1098.     ]
 ...
 [  -29.6303     10.9342    439.372   ...   -48.9162     17.8885
    702.     ]
 [  -30.8918     10.5933    452.372   ...   -29.9776     10.2135
    463.626  ]
 [  -30.6792     14.3615    458.277   ...   -30.7906     12.3865
    425.078  ]] shape (9000, 60) type <class 'numpy.ndarray'>
nparray_output_all [[ 1  1  1 ... -1 -1 -1]
 [-1 -1 -1 ...  1  1  1]
 [-1 -1 -1 ...  1  1  1]
 ...
 [-1  1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]] shape (9000, 20) t

...

nparray_input_all [[-3.15006e+01 -6.66272e+00 -4.50298e+02 ... -7.00543e+01 -1.27306e+01
  -9.58000e+02]
 [-3.33049e+01 -1.50220e+00 -4.82024e+02 ... -5.98097e+01 -2.99161e+00
  -8.17500e+02]
 [-7.16722e+01 -1.77983e+01 -1.09800e+03 ... -8.65402e+01 -1.97223e+01
  -1.29800e+03]
 ...
 [-3.10204e+01  8.31308e+00  4.54621e+02 ... -4.24032e+01  1.09858e+01
   5.97500e+02]
 [-3.08389e+01  1.15304e+01  4.27606e+02 ... -7.37304e+01  2.58513e+01
   1.10200e+03]
 [-3.14296e+01  1.05919e+00  4.47185e+02 ... -1.10557e+02  3.68484e+00
   1.49800e+03]] shape (9396, 60) type <class 'numpy.ndarray'>
nparray_output_all [[ 1  1  1 ...  1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 ...
 [-1 -1 -1 ...  1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 [ 1  1 -1 ... -1 -1  1]] shape (9396, 20) type <class 'numpy.ndarray'>
nparray_input_all [[-3.15006e+01 -6.66272e+00 -4.50298e+02 ... -7.00543e+01 -1.27306e+01
  -9.58000e+02]
 [-3.33049e+01 -1.50220e+00 -4.82024e+02 ... -5.98097e+01 -2.99161e+00
  -8.17500e

nparray_position [[ 3.92924e+01  6.77874e+00 -1.50250e+03]
 [ 8.37134e+01  8.01779e+00 -1.50250e+03]
 [ 6.23576e+01  1.00987e+01 -1.50250e+03]
 ...
 [-8.74072e+02 -2.30109e+00  2.95250e+03]
 [-9.08826e+02  6.00040e+01  2.95250e+03]
 [-8.89220e+02  4.03742e+01  2.95250e+03]]
nparray_input_all [[-3.27839e+01 -2.99231e+00 -4.36060e+02 ... -8.13627e+01 -1.54612e+00
  -1.09750e+03]
 [-2.98999e+01 -1.12410e+01 -4.55920e+02 ... -3.03465e+01 -9.96363e+00
  -4.49461e+02]
 [-3.41976e+01  1.05120e+00 -4.42061e+02 ... -1.11010e+02  1.20310e+00
  -1.49800e+03]
 ...
 [-3.15929e+01 -1.83620e+00  4.40078e+02 ... -6.08433e+01  3.52373e-01
   8.22500e+02]
 [-3.11819e+01  5.44967e+00  4.61067e+02 ... -5.68353e+01  1.14081e+01
   8.22000e+02]
 [-3.10286e+01  8.16763e+00  4.19998e+02 ... -3.09683e+01  9.23626e+00
   4.56109e+02]] shape (9622, 60) type <class 'numpy.ndarray'>
nparray_output_all [[-1  1  1 ... -1 -1 -1]
 [ 1  1 -1 ... -1 -1 -1]
 [-1 -1  1 ... -1 -1  1]
 ...
 [-1 -1 -1 ...  1 -1 -1]
 [ 1  1  

nparray_input_all [[-3.19668e+01 -5.32933e+00 -4.68884e+02 ... -7.35077e+01 -1.22300e+01
  -1.10250e+03]
 [-3.39063e+01  2.17944e-01 -4.59546e+02 ... -9.84488e+01 -1.45369e+00
  -1.30250e+03]
 [-3.21931e+01 -4.68200e+00 -4.55562e+02 ... -9.21331e+01 -1.64253e+01
  -1.29800e+03]
 ...
 [-3.15676e+01 -1.38692e+00  4.86400e+02 ... -9.01156e+01 -6.96686e+00
   1.30250e+03]
 [-3.10370e+01  8.01890e+00  4.24117e+02 ... -3.15323e+01  8.02997e+00
   4.04613e+02]
 [-3.06981e+01  1.40266e+01  4.64099e+02 ... -5.55021e+01  2.57228e+01
   8.18000e+02]] shape (9916, 60) type <class 'numpy.ndarray'>
nparray_output_all [[ 1 -1 -1 ...  1 -1  1]
 [ 1  1 -1 ...  1 -1 -1]
 [-1  1  1 ... -1  1 -1]
 ...
 [-1  1  1 ... -1 -1 -1]
 [ 1 -1 -1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]] shape (9916, 20) type <class 'numpy.ndarray'>
nparray_input_all [[-3.19668e+01 -5.32933e+00 -4.68884e+02 ... -7.35077e+01 -1.22300e+01
  -1.10250e+03]
 [-3.39063e+01  2.17944e-01 -4.59546e+02 ... -9.84488e+01 -1.45369e+00
  -1.30250e

nparray_position [[ 8.84164e+01  6.33159e+00 -1.50250e+03]
 [ 8.29015e+01  1.11638e+01 -1.50250e+03]
 [ 3.09301e+01  1.90762e+00 -1.50250e+03]
 ...
 [-9.37026e+02  1.08671e+02  2.95250e+03]
 [-9.77519e+02  1.27571e+02  2.95250e+03]
 [-9.72926e+02  3.49395e+01  2.95250e+03]]
nparray_input_all [[-2.95863e+01 -1.21377e+01 -4.39828e+02 ... -3.76260e+01 -1.92162e+01
  -5.97500e+02]
 [-3.14410e+01 -6.83309e+00 -4.75793e+02 ... -8.16088e+01 -1.76129e+01
  -1.29800e+03]
 [-3.26193e+01 -3.46318e+00 -4.85676e+02 ... -5.31516e+01 -5.61253e+00
  -8.22500e+02]
 ...
 [-2.88272e+01  1.26008e+01  4.25891e+02 ... -2.96520e+01  1.08891e+01
   4.22364e+02]
 [-3.15169e+01 -4.88346e-01  4.59264e+02 ... -5.53954e+01 -3.53964e+00
   8.17500e+02]
 [-3.14465e+01  7.59670e-01  4.46630e+02 ... -6.02966e+01 -8.58467e-01
   8.22500e+02]] shape (8062, 60) type <class 'numpy.ndarray'>
nparray_output_all [[ 1  1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1  1 -1]
 [-1  1 -1 ... -1 -1 -1]
 ...
 [ 1  1  1 ... -1 -1 -1]
 [-1 -1 -

nparray_input_all [[-3.38646e+01  9.87360e-02 -4.53971e+02 ... -4.36728e+01  1.30555e+00
  -5.97500e+02]
 [-2.93913e+01 -1.26955e+01 -4.43640e+02 ... -4.20593e+01 -1.74486e+01
  -6.02000e+02]
 [-2.93058e+01 -1.29401e+01 -4.36173e+02 ... -4.85599e+01 -2.14598e+01
  -6.97500e+02]
 ...
 [-3.14411e+01  8.55714e-01  4.84116e+02 ... -6.08002e+01 -1.95932e-01
   9.62500e+02]
 [-2.92212e+01  1.29236e+01  4.03161e+02 ... -4.18958e+01  1.86256e+01
   5.98000e+02]
 [-3.10214e+01  8.29598e+00  4.77264e+02 ... -5.50428e+01  1.09774e+01
   8.17500e+02]] shape (9011, 60) type <class 'numpy.ndarray'>
nparray_output_all [[ 1  1  1 ... -1 -1 -1]
 [ 1  1 -1 ... -1 -1 -1]
 [-1 -1 -1 ...  1 -1 -1]
 ...
 [ 1  1  1 ... -1 -1 -1]
 [-1 -1 -1 ... -1  1 -1]
 [-1  1  1 ... -1 -1 -1]] shape (9011, 20) type <class 'numpy.ndarray'>
nparray_input_all [[-3.38646e+01  9.87360e-02 -4.53971e+02 ... -4.36728e+01  1.30555e+00
  -5.97500e+02]
 [-2.93913e+01 -1.26955e+01 -4.43640e+02 ... -4.20593e+01 -1.74486e+01
  -6.02000e

In [25]:
# combine all numpy arrays into one
eventNumber="all"
fileNameNNInputTrainAll=outputFolderName+"/NN_2_data_Input_Train_"+eventNumber+".npy"
fileNameNNOutputTrainAll=outputFolderName+"/NN_2_data_Output_Train_"+eventNumber+".npy"
fileNameNNInputTestAll=outputFolderName+"/NN_2_data_Input_Test_"+eventNumber+".npy"
fileNameNNOutputTestAll=outputFolderName+"/NN_2_data_Output_Test_"+eventNumber+".npy"
    
for i,eventNumber in enumerate(list_eventNumber):
    fileNameNNInputTrain=outputFolderName+"/NN_2_data_Input_Train_"+eventNumber+".npy"
    fileNameNNOutputTrain=outputFolderName+"/NN_2_data_Output_Train_"+eventNumber+".npy"
    fileNameNNInputTest=outputFolderName+"/NN_2_data_Input_Test_"+eventNumber+".npy"
    fileNameNNOutputTest=outputFolderName+"/NN_2_data_Output_Test_"+eventNumber+".npy"
    
    nparray_Input_Train=np.load(fileNameNNInputTrain)
    nparray_Input_Test=np.load(fileNameNNInputTest)
    nparray_Output_Train=np.load(fileNameNNOutputTrain)
    nparray_Output_Test=np.load(fileNameNNOutputTest)
    
    if i==0:
        nparray_Input_Train_all=nparray_Input_Train
        nparray_Input_Test_all=nparray_Input_Test
        nparray_Output_Train_all=nparray_Output_Train
        nparray_Output_Test_all=nparray_Output_Test
    else:
        nparray_Input_Train_all=np.concatenate((nparray_Input_Train_all,nparray_Input_Train),axis=0,out=None)
        nparray_Input_Test_all=np.concatenate((nparray_Input_Test_all,nparray_Input_Test),axis=0,out=None)
        nparray_Output_Train_all=np.concatenate((nparray_Output_Train_all,nparray_Output_Train),axis=0,out=None)
        nparray_Output_Test_all=np.concatenate((nparray_Output_Test_all,nparray_Output_Test),axis=0,out=None)
    # done if
# done for loop 

print("nparray_Input_Train_all",nparray_Input_Train_all,"shape",nparray_Input_Train_all.shape,"type",type(nparray_Input_Train_all))
print("nparray_Input_Test_all",nparray_Input_Test_all,"shape",nparray_Input_Test_all.shape,"type",type(nparray_Input_Test_all))
print("nparray_Output_Train_all",nparray_Output_Train_all,"shape",nparray_Output_Train_all.shape,"type",type(nparray_Output_Train_all))
print("nparray_Output_Test_all",nparray_Output_Test_all,"shape",nparray_Output_Test_all.shape,"type",type(nparray_Output_Test_all))

np.save(fileNameNNInputTrainAll,nparray_Input_Train_all)
np.save(fileNameNNInputTestAll,nparray_Input_Test_all)
np.save(fileNameNNOutputTrainAll,nparray_Output_Train_all)
np.save(fileNameNNOutputTestAll,nparray_Output_Test_all)

nparray_Input_Train_all [[[-3.23306e+01]
  [-4.28872e+00]
  [-4.62092e+02]
  ...
  [-8.93000e+01]
  [-1.51782e+01]
  [-1.30250e+03]]

 [[-3.04775e+01]
  [-9.58902e+00]
  [-4.21256e+02]
  ...
  [-1.12212e+02]
  [-4.01146e+01]
  [-1.49800e+03]]

 [[-3.19316e+01]
  [-5.42998e+00]
  [-4.84890e+02]
  ...
  [-8.34928e+01]
  [-1.48256e+01]
  [-1.29800e+03]]

 ...

 [[-3.13739e+01]
  [ 2.04705e+00]
  [ 4.87235e+02]
  ...
  [-6.26886e+01]
  [ 6.31295e+00]
  [ 9.57500e+02]]

 [[-3.09550e+01]
  [ 9.47315e+00]
  [ 4.64051e+02]
  ...
  [-4.04887e+01]
  [ 1.09354e+01]
  [ 5.97500e+02]]

 [[-3.31662e+01]
  [-3.98620e-01]
  [ 4.46810e+02]
  ...
  [-7.94153e+01]
  [ 2.56644e-01]
  [ 1.09750e+03]]] shape (478540, 60, 1) type <class 'numpy.ndarray'>
nparray_Input_Test_all [[[  -30.0484 ]
  [  -10.8162 ]
  [ -427.981  ]
  ...
  [  -47.7968 ]
  [  -14.5214 ]
  [ -702.     ]]

 [[  -31.7544 ]
  [   -5.93686]
  [ -455.191  ]
  ...
  [ -102.996  ]
  [  -17.4721 ]
  [-1498.     ]]

 [[  -29.4708 ]
  [  -12.468

In [2]:
import numpy as np
l1=[1,2,3]
l2=[7,8,9]
l3=[5,4,6]


In [4]:
nparray_l1=np.array(l1)
nparray_l1

array([1, 2, 3])

In [5]:
nparray_l2=np.array(l2)
nparray_l2

array([7, 8, 9])

In [6]:
nparray_l3=np.array(l3)
nparray_l3

array([5, 4, 6])

In [13]:
n=np.concatenate((l1,l2,l3),axis=0,out=None)
n

array([1, 2, 3, 7, 8, 9, 5, 4, 6])

In [10]:
list_nparray=[nparray_l1,nparray_l2,nparray_l3]
list_nparray

[array([1, 2, 3]), array([7, 8, 9]), array([5, 4, 6])]

In [18]:
for i,nparray in enumerate(list_nparray):
    if i==0:
        nparray_all=nparray
    else:
        nparray_all=np.concatenate((nparray_all,nparray),axis=0,out=None)
print(nparray_all.shape,nparray_all.dtype,type(nparray_all))

(9,) int64 <class 'numpy.ndarray'>
